itThese notebooks from d2l.ai contain some information about Recurrent Neural Networks in general and will give a more detailed look into the LSTM architecture. This material is for you to review. The mandatory exercises can be found after each section.

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# Recurrent Neural Networks
:label:`sec_plain_rnn`


In :numref:`sec_language_model` we introduced $n$-gram models, where the conditional probability of word $x_t$ at position $t$ only depends on the $n-1$ previous words. If we want to check the possible effect of words earlier than $t-(n-1)$ on $x_t$, we need to increase $n$. However, the number of model parameters would also increase exponentially with it, as we need to store $|V|^n$ numbers for a vocabulary $V$. Hence, rather than modeling $p(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$ it is preferable to use a *latent variable model* in which we have

$$p(x_t \mid x_{t-1}, \ldots, x_1) \approx p(x_t \mid x_{t-1}, h_{t}).$$

Here $h_t$ is a *latent variable* that stores the sequence information. A latent variable is also called as *hidden variable*, *hidden state* or *hidden state variable*. The hidden state at time $t$ could be computed based on both input $x_{t}$ and hidden state $h_{t-1}$, that is

$$h_t = f(x_{t}, h_{t-1}).$$

For a sufficiently powerful function $f$, the latent variable model is not an approximation. After all, $h_t$ could simply store all the data it observed so far. We discussed this in :numref:`sec_sequence`. But it could potentially makes both computation and storage expensive.

Note that we also use $h$ to denote the number of hidden units of a hidden layer. Hidden layers and hidden states refer to two very different concepts. Hidden layers are, as explained, layers that are hidden from view on the path from input to output. Hidden states are technically speaking *inputs* to whatever we do at a given step. Instead, they can only be computed by looking at data at previous iterations. In this sense they have much in common with latent variable models in statistics, such as clustering or topic models where the clusters affect the output but cannot be directly observed.

Recurrent neural networks are neural networks with hidden states. Before introducing this model, let us first revisit the multi-layer perceptron introduced in :numref:`sec_mlp`.

## Recurrent Networks Without Hidden States

Let us take a look at a multilayer perceptron with a single hidden layer. Given a minibatch of the instances $\mathbf{X} \in \mathbb{R}^{n \times d}$ with sample size $n$ and $d$ inputs. Let the hidden layer's activation function be $\phi$. Hence, the hidden layer's output $\mathbf{H} \in \mathbb{R}^{n \times h}$ is calculated as

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).$$

:eqlabel:`rnn_h_without_state`

Here, we have the weight parameter $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$, bias parameter $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$, and the number of hidden units $h$, for the hidden layer.

The hidden variable $\mathbf{H}$ is used as the input of the output layer. The output layer is given by

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q.$$

Here, $\mathbf{O} \in \mathbb{R}^{n \times q}$ is the output variable, $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ is the weight parameter, and $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ is the bias parameter of the output layer.  If it is a classification problem, we can use $\text{softmax}(\mathbf{O})$ to compute the probability distribution of the output category.



This is entirely analogous to the regression problem we solved previously in :numref:`sec_sequence`, hence we omit details. Suffice it to say that we can pick $(x_t, x_{t-1})$ pairs at random and estimate the parameters $\mathbf{W}$ and $\mathbf{b}$ of our network via autograd and stochastic gradient descent.

## Recurrent Networks with Hidden States

Matters are entirely different when we have hidden states. Let us look at the structure in some more detail. Remember that we often call iteration $t$ as time $t$ in an optimization algorithm, time in a recurrent neural network refers to steps within an iteration. Assume that we have $\mathbf{X}_t \in \mathbb{R}^{n \times d}$, $t=1,\ldots, T$, in an iteration. And $\mathbf{H}_t  \in \mathbb{R}^{n \times h}$ is the hidden variable of timestep $t$ from the sequence.  Unlike the multilayer perceptron, here we save the hidden variable $\mathbf{H}_{t-1}$ from the previous timestep and introduce a new weight parameter $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$, to describe how to use the hidden variable of the previous timestep in the current timestep. Specifically, the calculation of the hidden variable of the current timestep is determined by the input of the current timestep together with the hidden variable of the previous timestep:

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$

Compared with :eqref:`rnn_h_without_state`, we added one more $\mathbf{H}_{t-1} \mathbf{W}_{hh}$ here. From the relationship between hidden variables $\mathbf{H}_t$ and $\mathbf{H}_{t-1}$ of adjacent timesteps, we know that those variables captured and retained the sequence's historical information up to the current timestep, just like the state or memory of the neural network's current timestep. Therefore, such a hidden variable is called a *hidden state*. Since the hidden state uses the same definition of the previous timestep in the current timestep, the computation of the equation above is recurrent, hence the name recurrent neural network (RNN).

There are many different RNN construction methods.  RNNs with a hidden state defined by the equation above are very common. For timestep $t$, the output of the output layer is similar to the computation in the multilayer perceptron:

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

RNN parameters include the weight $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$ of the hidden layer with the bias $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$, and the weight $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ of the output layer with the bias $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$. It is worth mentioning that RNNs always use these model parameters, even for different timesteps. Therefore, the number of RNN model parameters does not grow as the number of timesteps increases.

:numref:`fig_rnn` shows the computational logic of an RNN at three adjacent timesteps. In timestep $t$, the computation of the hidden state can be treated as an entry of a fully connected layer with the activation function $\phi$ after concatenating the input $\mathbf{X}_t$ with the hidden state $\mathbf{H}_{t-1}$ of the previous timestep.  The output of the fully connected layer is the hidden state of the current timestep $\mathbf{H}_t$. Its model parameter is the concatenation of $\mathbf{W}_{xh}$ and $\mathbf{W}_{hh}$, with a bias of $\mathbf{b}_h$. The hidden state of the current timestep $t$, $\mathbf{H}_t$, will participate in computing the hidden state $\mathbf{H}_{t+1}$ of the next timestep $t+1$. What is more, $\mathbf{H}_t$ will become the input for $\mathbf{O}_t$, the fully connected output layer of the current timestep.

![An RNN with a hidden state. ](http://d2l.ai/_images/rnn.svg)

:label:`fig_rnn`

## Steps in a Language Model

Now we illustrate how RNNs can be used to build a language model. For simplicity of illustration we use words rather than characters as the inputs, since the former are easier to comprehend. Let the minibatch size be 1, and the sequence of the text be the beginning of our dataset, i.e., "the time machine by H. G. Wells". :numref:`fig_rnn_train` illustrates how to estimate the next word based on the present and previous words. During the training process, we run a softmax operation on the output from the output layer for each timestep, and then use the cross-entropy loss function to compute the error between the result and the label. Due to the recurrent computation of the hidden state in the hidden layer, the output of timestep 3, $\mathbf{O}_3$, is determined by the text sequence "the", "time", and "machine" respectively. Since the next word of the sequence in the training data is "by", the loss of timestep 3 will depend on the probability distribution of the next word generated based on the feature sequence "the", "time", "machine" and the label "by" of this timestep.

![Word-level RNN language model. The input and label sequences are `the time machine by H.` and `time machine by H. G.` respectively. ](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/rnn-train.svg?raw=1)
:label:`fig_rnn_train`

In practice, each word is presented by a $d$ dimensional vector, and we use a batch size $n>1$. Therefore, the input $\mathbf X_t$ at timestep $t$ will be a $n\times d$ matrix, which is identical to what we discussed before.

## Perplexity

Last, let us discuss about how to measure the sequence model quality. One way is to check how surprising the text is. A good language model is able to predict with high accuracy tokens that what we will see next. Consider the following continuations of the phrase "It is raining", as proposed by different language models:

1. "It is raining outside"
1. "It is raining banana tree"
1. "It is raining piouw;kcj pwepoiut"

In terms of quality, example 1 is clearly the best. The words are sensible and logically coherent. While it might not quite accurately reflect which word follows semantically ("in San Francisco" and "in winter" would have been perfectly reasonable extensions), the model is able to capture which kind of word follows. Example 2 is considerably worse by producing a nonsensical extension. Nonetheless, at least the model has learned how to spell words and some degree of correlation between words. Last, example 3 indicates a poorly trained model that does not fit data properly.

We might measure the quality of the model by computing $p(w)$, i.e., the likelihood of the sequence. Unfortunately this is a number that is hard to understand and difficult to compare. After all, shorter sequences are much more likely to occur than the longer ones, hence evaluating the model on Tolstoy's magnum opus ["War and Peace"](https://www.gutenberg.org/files/2600/2600-h/2600-h.htm) will inevitably produce a much smaller likelihood than, say, on Saint-Exupery's novella ["The Little Prince"](https://en.wikipedia.org/wiki/The_Little_Prince). What is missing is the equivalent of an average.

Information theory comes handy here and we will introduce more in :numref:`sec_information_theory`. If we want to compress text, we can ask about estimating the next symbol given the current set of symbols. A lower bound on the number of bits is given by $-\log_2 p(x_t \mid x_{t-1}, \ldots, x_1)$. A good language model should allow us to predict the next word quite accurately. Thus, it should allow us to spend very few bits on compressing the sequence. So we can measure it by the average number of bits that we need to spend.

$$\frac{1}{n} \sum_{t=1}^n -\log p(x_t \mid x_{t-1}, \ldots, x_1).$$

This makes the performance on documents of different lengths comparable. For historical reasons, scientists in natural language processing prefer to use a quantity called *perplexity* rather than bitrate. In a nutshell, it is the exponential of the above:

$$\mathrm{PPL} := \exp\left(-\frac{1}{n} \sum_{t=1}^n \log p(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

It can be best understood as the harmonic mean of the number of real choices that we have when deciding which word to pick next. Note that perplexity naturally generalizes the notion of the cross-entropy loss defined when we introduced the softmax regression (:numref:`sec_softmax`). That is, for a single symbol both definitions are identical bar the fact that one is the exponential of the other. Let us look at a number of cases:

* In the best case scenario, the model always estimates the probability of the next symbol as $1$. In this case the perplexity of the model is $1$.
* In the worst case scenario, the model always predicts the probability of the label category as 0. In this situation, the perplexity is infinite.
* At the baseline, the model predicts a uniform distribution over all tokens. In this case, the perplexity equals the size of the dictionary `len(vocab)`. In fact, if we were to store the sequence without any compression, this would be the best we could do to encode it. Hence, this provides a nontrivial upper bound that any model must satisfy.


## Summary

* A network that uses recurrent computation is called a recurrent neural network (RNN).
* The hidden state of the RNN can capture historical information of the sequence up to the current timestep.
* The number of RNN model parameters does not grow as the number of timesteps increases.
* We can create language models using a character-level RNN.

---

**Exercise**: Implement a simple time series forecasting model. For this, we have built an ECG dataset from a prototypical heartbeat originating from the dataset ECGFiveDays by Y. Chen and E. Keogh published on www.timeseriesclassification.com/description.php?Dataset=ECGFiveDays. This way, you don't have to download anything.
Use an RNN here.

In [35]:
# data
prototype = \
np.array([ 8.4608798e-02,  1.8681801e-02,  1.3610494e-02,  5.4180953e-02,
        3.4678795e-03,  4.4038339e-02,  3.8967031e-02,  4.4038339e-02,
        6.9394876e-02,  1.2517926e-01,  1.8603495e-01,  2.4181933e-01,
        2.5196194e-01,  3.1281763e-01,  2.7731848e-01,  1.3025057e-01,
       -6.6747353e-03, -6.2459116e-02, -1.6388526e-01, -1.2838611e-01,
       -1.5881396e-01, -1.9938442e-01, -1.9938442e-01, -1.6388526e-01,
       -1.5881396e-01, -1.7402788e-01, -1.4360004e-01, -1.6388526e-01,
        1.1503665e-01,  1.6161436e+00,  3.8069485e+00,  1.9609925e+00,
       -3.0139600e+00, -6.3914507e+00, -5.4076171e+00, -3.4145933e+00,
       -1.7917750e+00, -7.6737084e-01, -4.2252195e-01, -2.5009749e-01,
       -1.6388526e-01, -8.2744347e-02, -8.2744347e-02, -4.2173887e-02,
        8.5391868e-03,  3.8967031e-02,  3.3895724e-02,  1.0996534e-01,
        1.4039318e-01,  2.8746109e-01,  3.7367332e-01,  5.2581254e-01,
        7.3373615e-01,  9.4165975e-01,  1.1647973e+00,  1.3524356e+00,
        1.4741470e+00,  1.5654305e+00,  1.5705019e+00,  1.4386479e+00,
        1.1445121e+00,  8.5037621e-01,  5.9173954e-01,  3.3310286e-01,
        2.1646279e-01,  1.0996534e-01,  4.9109646e-02, -1.6817350e-02,
       -2.6959965e-02, -6.7530424e-02, -1.1746043e-02, -2.1888657e-02,
       -2.6959965e-02,  1.8681801e-02,  8.5391868e-03,  2.8824417e-02,
        3.3895724e-02,  2.3753110e-02,  5.9252261e-02,  3.8967031e-02])

# create longer sequence
elongated = np.array([*prototype]*20)
elongated_noised = elongated + np.random.random(elongated.shape[0])

# splitting sequence
test_train_split = int(0.8*elongated_noised.shape[0])
train_raw = elongated_noised[:test_train_split]
test_raw = elongated_noised[test_train_split:]

# standardizing
train_seq = torch.FloatTensor((train_raw - train_raw.mean()) / train_raw.std())
test_seq = torch.FloatTensor((test_raw - train_raw.mean()) / train_raw.std())

# building train set
train_sequences = []
pattern_length = prototype.shape[0]
for i in range(train_seq.shape[0]-pattern_length):
    seq = train_seq[i:i+pattern_length]
    label = train_seq[i+pattern_length:i+pattern_length+1]
    train_sequences.append((seq, label))

In [3]:
len(train_sequences[0][0])
train_seq
#torch.FloatTensor(train_sequences)
len(train_sequences)

1200

We provide a small skeleton that might help to solve the exercise a little faster (by filling the empty lines that have a #). But you can also choose to ignore it and use your own structure. In any case, you should implement some additional functionality to analyze the results.

Describe your observations.
* How far into the future can you reliably predict? What influence do the number of epochs and hidden size have?

And finally, plot your predictions.

In [4]:
# build model architecture
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size,hidden_size)
        self.linear = nn.Linear(hidden_size,output_size)
        
        self.hidden_state = self._init_hidden(1)
    
    def forward(self, input_seq):
        #
        # 
        #

        # input of shape (seq_len, batch, input_size)
        output , hidden = self.rnn(input_seq.view(len(input_seq) ,1, -1),self.hidden_state)
        predictions = self.linear(output.view(len(input_seq), -1))
        return predictions[-1]
    
    def _init_hidden(self, batch_size):
      #h_0 of shape (num_layers * num_directions, batch, hidden_size)
      hidden = torch.zeros(1,batch_size,self.hidden_size)
      return hidden

In [5]:
# initialize model
rnn_model = RNN(1,100,1)
print(rnn_model)
lr = 0.01

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=lr)

RNN(
  (rnn): RNN(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [6]:
# training routine
epochs = 2

for i in range(epochs):
    for seq, labels in train_sequences:
        optimizer.zero_grad()
        #rnn_model.hidden_state = rnn_model._init_hidden(1)

        y_pred = rnn_model(seq)

        loss = loss_function(y_pred,label)
        loss.backward()
        optimizer.step()

    if i%1 == 0:
        print(f'epoch: {i:3} - loss: {loss.item():10.8f}')

print(f'epoch: {i:3} - loss: {loss.item():10.8f}')

epoch:   0 - loss: 0.00019393
epoch:   1 - loss: 0.00194608
epoch:   1 - loss: 0.00194608


In [7]:
pattern_length

80

In [8]:
# prediction loop
rnn_model.eval()
seq = train_seq[-pattern_length:]
new_seq = seq
for i in range(test_seq.shape[0]):
    with torch.no_grad():
        #rnn_model.hidden_state = rnn_model._init_hidden(1)
        
        # use output of model to build the new sequence for prediction
        new_seq=torch.cat((new_seq,rnn_model(seq)),0)
        
        seq = new_seq[-pattern_length:]
predictions = new_seq[-test_seq.shape[0]:]
predictions

TypeError: expected Tensor as element 1 in argument 0, but got float

In [ ]:
seq

In [ ]:
# plotting and answering questions

---

# Long Short Term Memory (LSTM)
:label:`sec_lstm`

The challenge to address long-term information preservation and short-term input
skipping in latent variable models has existed for a long time. One of the
earliest approaches to address this was the LSTM
:cite:`Hochreiter.Schmidhuber.1997`.  It shares many of the properties of the
Gated Recurrent Unit (GRU). Interestingly, LSTM's design is slightly more complex than GRU but predates GRU by almost two decades.

Arguably it is inspired by logic gates of a computer. To control a memory cell
we need a number of gates. One gate is needed to read out the entries from the
cell (as opposed to reading any other cell). We will refer to this as the
*output* gate. A second gate is needed to decide when to read data into the
cell. We refer to this as the *input* gate. Last, we need a mechanism to reset
the contents of the cell, governed by a *forget* gate. The motivation for such a
design is the same as before, namely to be able to decide when to remember and
when to ignore inputs in the latent state via a dedicated mechanism. Let us see
how this works in practice.

## Gated Memory Cells

Three gates are introduced in LSTMs: the input gate, the forget gate, and the output gate. In addition to that we will introduce the memory cell that has the same shape as the hidden state. Strictly speaking this is just a fancy version of a hidden state, engineered to record additional information.

### Input Gates, Forget Gates, and Output Gates

Just like with GRUs, the data feeding into the LSTM gates is the input at the current timestep $\mathbf{X}_t$ and the hidden state of the previous timestep $\mathbf{H}_{t-1}$. These inputs are processed by a fully connected layer and a sigmoid activation function to compute the values of input, forget and output gates. As a result, the three gates' all output values are in the range of $[0, 1]$. :numref:`lstm_0` illustrates the data flow for the input, forget, and output gates.

![Calculation of input, forget, and output gates in an LSTM. ](http://d2l.ai/_images/lstm_0.svg)
:label:`lstm_0`

We assume that there are $h$ hidden units, the minibatch is of size $n$, and number of inputs is $d$. Thus, the input is $\mathbf{X}_t \in \mathbb{R}^{n \times d}$ and the hidden state of the last timestep is $\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$. Correspondingly, the gates are defined as follows: the input gate is $\mathbf{I}_t \in \mathbb{R}^{n \times h}$, the forget gate is $\mathbf{F}_t \in \mathbb{R}^{n \times h}$, and the output gate is $\mathbf{O}_t \in \mathbb{R}^{n \times h}$. They are calculated as follows:

$$
\begin{aligned}
\mathbf{I}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xi} + \mathbf{H}_{t-1} \mathbf{W}_{hi} + \mathbf{b}_i),\\
\mathbf{F}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xf} + \mathbf{H}_{t-1} \mathbf{W}_{hf} + \mathbf{b}_f),\\
\mathbf{O}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xo} + \mathbf{H}_{t-1} \mathbf{W}_{ho} + \mathbf{b}_o),
\end{aligned}
$$

where $\mathbf{W}_{xi}, \mathbf{W}_{xf}, \mathbf{W}_{xo} \in \mathbb{R}^{d \times h}$ and $\mathbf{W}_{hi}, \mathbf{W}_{hf}, \mathbf{W}_{ho} \in \mathbb{R}^{h \times h}$ are weight parameters and $\mathbf{b}_i, \mathbf{b}_f, \mathbf{b}_o \in \mathbb{R}^{1 \times h}$ are bias parameters.


### Candidate Memory Cell

Next we design the memory cell. Since we have not specified the action of the various gates yet, we first introduce the *candidate* memory cell $\tilde{\mathbf{C}}_t \in \mathbb{R}^{n \times h}$. Its computation is similar to the three gates described above, but using a $\tanh$ function with a value range for $[-1, 1]$ as the activation function. This leads to the following equation at timestep $t$.

$$\tilde{\mathbf{C}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xc} + \mathbf{H}_{t-1} \mathbf{W}_{hc} + \mathbf{b}_c).$$

Here $\mathbf{W}_{xc} \in \mathbb{R}^{d \times h}$ and $\mathbf{W}_{hc} \in \mathbb{R}^{h \times h}$ are weight parameters and $\mathbf{b}_c \in \mathbb{R}^{1 \times h}$ is a bias parameter.

A quick illustration of the candidate memory cell is shown in :numref:`lstm_1`.

![Computation of candidate memory cells in LSTM. ](http://d2l.ai/_images/lstm_1.svg)
:label:`lstm_1`


### Memory Cell

In GRUs, we had a single mechanism to govern input and forgetting. Here in LSTMs we have two parameters, $\mathbf{I}_t$ which governs how much we take new data into account via $\tilde{\mathbf{C}}_t$ and the forget parameter $\mathbf{F}_t$ which addresses how much of the old memory cell content $\mathbf{C}_{t-1} \in \mathbb{R}^{n \times h}$ we retain. Using the same pointwise multiplication trick as before, we arrive at the following update equation.

$$\mathbf{C}_t = \mathbf{F}_t \odot \mathbf{C}_{t-1} + \mathbf{I}_t \odot \tilde{\mathbf{C}}_t.$$

If the forget gate is always approximately $1$ and the input gate is always approximately $0$, the past memory cells $\mathbf{C}_{t-1}$ will be saved over time and passed to the current timestep. This design was introduced to alleviate the vanishing gradient problem and to better capture dependencies for time series with long range dependencies. We thus arrive at the flow diagram in :numref:`lstm_2`.

![Computation of memory cells in an LSTM. Here, the multiplication is carried out elementwise. ](http://d2l.ai/_images/lstm_2.svg)

:label:`lstm_2`


### Hidden States

Last, we need to define how to compute the hidden state $\mathbf{H}_t \in \mathbb{R}^{n \times h}$. This is where the output gate comes into play. In LSTM it is simply a gated version of the $\tanh$ of the memory cell. This ensures that the values of $\mathbf{H}_t$ are always in the interval $(-1, 1)$. Whenever the output gate is $1$ we effectively pass all memory information through to the predictor, whereas for output $0$ we retain all the information only within the memory cell and perform no further processing. :numref:`lstm_3` has a graphical illustration of the data flow.

$$\mathbf{H}_t = \mathbf{O}_t \odot \tanh(\mathbf{C}_t).$$

![Computation of the hidden state. Multiplication is elementwise. ](http://d2l.ai/_images/lstm_3.svg)
:label:`lstm_3`

# Summary
• LSTMs have three types of gates: input, forget and output gates which control the flow of information.

• The hidden layer output of LSTM includes hidden states and memory cells. Only hidden states are passed into the output layer. Memory cells are entirely internal.

• LSTMs can help cope with vanishing and exploding gradients due to long range dependencies and short-range irrelevant data.

• In many cases LSTMs perform slightly better than GRUs but they are more costly to train and execute due to the larger latent state size.

• LSTMs are the prototypical latent variable autoregressive model with nontrivial state control. Many variants thereof have been proposed over the years, e.g. 
multiple layers, residual connections, different types of regularization.

• Training LSTMs and other sequence models is quite costly due to the long dependency of the sequence. Later we will encounter alternative models such as transformers that can be used in some cases.

# References
[1] Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural computation, 9(8), 1735- 1780.



---

**Exercise**: You already implemented an RNN model. Now we'd like you to implement a forecasting model that uses LSTM architecture. Keep in mind that the overall structure is very similar to the exercise above. You may reuse large parts but be aware of the changing hidden state.

Again, monitor the impact of epochs and try out different hidden dimensionalities. Describe your observations and plot the predictions.

Compare these results with the RNN model and explain differences.

In [ ]:
# Implementation

In [ ]:
# Observations and Plots

In [ ]:
# example for plotting predictions
plt.figure(figsize=(14, 6))
plt.grid(True)
plt.plot(elongated_noised)
plt.plot(np.arange(test_train_split, elongated_noised.shape[0]), predictions)
plt.show()

---